In [39]:
import pandas as pd
import numpy as np
import sympy
import scipy
import sklearn.metrics as metrics
import statsmodels.api as sm
data = pd.read_csv('Purchase_Likelihood.csv')




In [40]:
data.head()

group_size  homeowner  married_couple  A
0           2          0               1  1
1           2          0               1  1
2           2          0               1  1
3           2          0               1  1
4           2          0               1  1

In [41]:
def create_interaction (inDF1, inDF2):
    name1 = inDF1.columns
    name2 = inDF2.columns
    outDF = pd.DataFrame()
    for col1 in name1:
        for col2 in name2:
            outName = col1 + " * " + col2
            outDF[outName] = inDF1[col1] * inDF2[col2]
    return(outDF)

# A function that find the non-aliased columns, fit a logistic model, and return the full parameter estimates
def build_mnlogit (fullX, y, debug = 'N'):
    # Number of all parameters
    nFullParam = fullX.shape[1]

    # Number of target categories
    y_category = y.cat.categories
    nYCat = len(y_category)

    # Find the non-redundant columns in the design matrix fullX
    reduced_form, inds = sympy.Matrix(fullX.values).rref()

    # These are the column numbers of the non-redundant columns
    if (debug == 'Y'):
        print('Column Numbers of the Non-redundant Columns:')
        print(inds)

    # Extract only the non-redundant columns for modeling
    X = fullX.iloc[:, list(inds)]

    # The number of free parameters
    thisDF = len(inds) * (nYCat - 1)

    # Build a multionomial logistic model
    logit = sm.MNLogit(y, X)
    thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
    thisParameter = thisFit.params
    thisLLK = logit.loglike(thisParameter.values)

    if (debug == 'Y'):
        print(thisFit.summary())
        print("Model Parameter Estimates:\n", thisParameter)
        print("Model Log-Likelihood Value =", thisLLK)
        print("Number of Free Parameters =", thisDF)

    # Recreat the estimates of the full parameters
    workParams = pd.DataFrame(np.zeros(shape = (nFullParam, (nYCat - 1))))
    workParams = workParams.set_index(keys = fullX.columns)
    fullParams = pd.merge(workParams, thisParameter, how = "left", left_index = True, right_index = True)
    fullParams = fullParams.drop(columns = '0_x').fillna(0.0)

    # Return model statistics
    return (thisLLK, thisDF, fullParams)


In [42]:
data = data.dropna()

In [43]:
y = data['A'].astype('category')

significanceList = []
dof = []
chi_sq_stats = []
# group_size, homeowner, married_couple, group_size * homeowner, and homeowner * married_couple 
# Specify JOB and REASON as categorical variables
xI = pd.get_dummies(data[['group_size']].astype('category'))
xJ = pd.get_dummies(data[['homeowner']].astype('category'))
xK = pd.get_dummies(data[['married_couple']].astype('category'))


In [44]:
# Intercept only
designX = pd.DataFrame(y.where(y.isnull(), 1))
LLK0, DF0, fullParams0 = build_mnlogit (designX, y, debug = 'Y')


Column Numbers of the Non-redundant Columns:
(0,)
Optimization terminated successfully.
         Current function value: 0.895013
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665247
Method:                           MLE   Df Model:                            0
Date:                Wed, 30 Oct 2019   Pseudo R-squ.:               6.440e-11
Time:                        17:06:22   Log-Likelihood:            -5.9541e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                       nan
       A=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
A              1.0869      0.003    356.296      0.000    

In [45]:
# Intercept + group_size
designX = sm.add_constant(xI, prepend=True)
LLK_1R, DF_1R, fullParams_1R = build_mnlogit (designX, y, debug = 'Y')
testDev = 2 * (LLK_1R - LLK0)
testDF = DF_1R - DF0
testPValue = scipy.stats.chi2.sf(testDev, testDF)
print('Deviance Chi=Square Test')
print('Chi-Square Statistic = ', testDev)
print('  Degreee of Freedom = ', testDF)
print('        Significance = ', testPValue)
chi_sq_stats.append(testDev)
dof.append(testDF)
significanceList.append(testPValue)

D:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3)
Optimization terminated successfully.
         Current function value: 0.894271
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665241
Method:                           MLE   Df Model:                            6
Date:                Wed, 30 Oct 2019   Pseudo R-squ.:               0.0008293
Time:                        17:09:36   Log-Likelihood:            -5.9491e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                4.348e-210
         A=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.5446      0.085      6.441 

In [46]:
# Intercept + group_size + homeowner
designX = xI
designX = designX.join(xJ)
designX = sm.add_constant(designX, prepend=True)
LLK_1R_1J, DF_1R_1J, fullParams_1R_1J = build_mnlogit (designX, y, debug = 'Y')
testDev = 2 * (LLK_1R_1J - LLK_1R)
testDF = DF_1R_1J - DF_1R
testPValue = scipy.stats.chi2.sf(testDev, testDF)
print('Deviance Chi=Square Test')
print('Chi-Square Statistic = ', testDev)
print('  Degreee of Freedom = ', testDF)
print('        Significance = ', testPValue)
chi_sq_stats.append(testDev)
dof.append(testDF)
significanceList.append(testPValue)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5)
Optimization terminated successfully.
         Current function value: 0.889861
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665239
Method:                           MLE   Df Model:                            8
Date:                Wed, 30 Oct 2019   Pseudo R-squ.:                0.005757
Time:                        17:14:45   Log-Likelihood:            -5.9198e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
         A=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.6231      0.085      7.3

In [47]:
# Intercept + group_size + homeowner + married_couple
designX = xI
designX = designX.join(xJ)
designX = designX.join(xK)
designX = sm.add_constant(designX, prepend=True)
LLK_1R_1J, DF_1R_1J, fullParams_1R_1J = build_mnlogit (designX, y, debug = 'Y')
testDev = 2 * (LLK_1R_1J - LLK_1R)
testDF = DF_1R_1J - DF_1R
testPValue = scipy.stats.chi2.sf(testDev, testDF)
print('Deviance Chi=Square Test')
print('Chi-Square Statistic = ', testDev)
print('  Degreee of Freedom = ', testDF)
print('        Significance = ', testPValue)
chi_sq_stats.append(testDev)
dof.append(testDF)
significanceList.append(testPValue)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5, 7)
Optimization terminated successfully.
         Current function value: 0.889797
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665237
Method:                           MLE   Df Model:                           10
Date:                Wed, 30 Oct 2019   Pseudo R-squ.:                0.005828
Time:                        17:20:37   Log-Likelihood:            -5.9194e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
             A=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.6282     

In [48]:
# Intercept + group_size + homeowner + married_couple + group_size * homeowner
designX = xI
designX = designX.join(xJ)
designX = designX.join(xK)
# Create the columns for the group_size * homeowner interaction effect
xIJ = create_interaction(xI, xJ)
designX = designX.join(xIJ)
# xJK = create_interaction(xJ,xK)
designX = sm.add_constant(designX, prepend=True)
LLK_2RJ, DF_2RJ, fullParams_2RJ = build_mnlogit (designX, y, debug = 'Y')
testDev = 2 * (LLK_2RJ - LLK_1R_1J)
testDF = DF_2RJ - DF_1R_1J
testPValue = scipy.stats.chi2.sf(testDev, testDF)
print('Deviance Chi=Square Test')
print('Chi-Square Statistic = ', testDev)
print('  Degreee of Freedom = ', testDF)
print('        Significance = ', testPValue)
chi_sq_stats.append(testDev)
dof.append(testDF)
significanceList.append(testPValue)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5, 7, 9, 11, 13)
Optimization terminated successfully.
         Current function value: 0.889606
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665231
Method:                           MLE   Df Model:                           16
Date:                Wed, 30 Oct 2019   Pseudo R-squ.:                0.006041
Time:                        17:30:48   Log-Likelihood:            -5.9181e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                       A=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
c

In [49]:
# Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple
designX = xI
designX = designX.join(xJ)
designX = designX.join(xK)

# Create the columns for the group_size * homeowner interaction effect
xIJ = create_interaction(xI, xJ)
designX = designX.join(xIJ)
xJK = create_interaction(xJ,xK)
designX = designX.join(xJK)

designX = sm.add_constant(designX, prepend=True)
LLK_2RJ, DF_2RJ, fullParams_2RJ = build_mnlogit (designX, y, debug = 'Y')
testDev = 2 * (LLK_2RJ - LLK_1R_1J)
testDF = DF_2RJ - DF_1R_1J
testPValue = scipy.stats.chi2.sf(testDev, testDF)
print('Deviance Chi=Square Test')
print('Chi-Square Statistic = ', testDev)
print('  Degreee of Freedom = ', testDF)
print('        Significance = ', testPValue)
chi_sq_stats.append(testDev)
dof.append(testDF)
significanceList.append(testPValue)

Column Numbers of the Non-redundant Columns:
(0, 1, 2, 3, 5, 7, 9, 11, 13, 17)
Optimization terminated successfully.
         Current function value: 0.889553
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                      A   No. Observations:               665249
Model:                        MNLogit   Df Residuals:                   665229
Method:                           MLE   Df Model:                           18
Date:                Wed, 30 Oct 2019   Pseudo R-squ.:                0.006101
Time:                        17:43:09   Log-Likelihood:            -5.9177e+05
converged:                       True   LL-Null:                   -5.9541e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           A=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [50]:
# ALl except => (0, 1, 2, 3, 5, 7, 9, 11, 13, 17) => alias parameters
# Degree of Freedom => 8
# c) chi sq test stats , signifcance , degree of freedom
# d) table values for feature importance index


import math
index_vals = [
              'Intercept + group_size',
              'Intercept + group_size + homeowner',
              'Intercept + group_size + homeowner + married_couple',
              'Intercept + group_size + homeowner + married_couple + group_size * homeowner',
              'Intercept + group_size + homeowner + married_couple + group_size * homeowner + homeowner * married_couple']

logs = [-math.log(s,10) if s!=0 else 0 for s in significanceList]
d = {'Chi Square Test':chi_sq_stats,'Significance':significanceList,'Degrees Of Freedom':dof,'Feature Importance Index':logs}
dataframe = pd.DataFrame(d,index=index_vals)
dataframe

Chi Square Test  \
Intercept + group_size                                   987.576601   
Intercept + group_size + homeowner                      5867.781500   
Intercept + group_size + homeowner + married_co...      5952.359503   
Intercept + group_size + homeowner + married_co...       254.078125   
Intercept + group_size + homeowner + married_co...       324.920402   

                                                     Significance  \
Intercept + group_size                              4.347870e-210   
Intercept + group_size + homeowner                   0.000000e+00   
Intercept + group_size + homeowner + married_co...   0.000000e+00   
Intercept + group_size + homeowner + married_co...   5.512106e-52   
Intercept + group_size + homeowner + married_co...   2.025652e-65   

                                                    Degrees Of Freedom  \
Intercept + group_size                                               6   
Intercept + group_size + homeowner                                   2   
Intercept + group_size + homeowner + married_co...                   4   
Intercept + group_size + homeowner + married_co...                   6   
Intercept + group_size + homeowner + married_co...                   8   

                                                    Feature Importance Index  
Intercept + group_size                                            209.361723  
Intercept + group_size + homeowner                                  0.000000  
Intercept + group_size + homeowner + married_co...                  0.000000  
Intercept + group_size + homeowner + married_co...                 51.258682  
Intercept + group_size + homeowner + married_co...                 64.693435

In [51]:
# (0, 1, 2, 3, 5, 7, 9, 11, 13, 17)
X = [0,1, 2, 3, 5, 7,9, 11, 13, 17]
designX.loc[:, ~designX.columns.isin([list(designX.columns)[i] for i in range(0,len(designX)) if i in X])].head()
# designX.head()



group_size_4  homeowner_1  married_couple_1  group_size_1 * homeowner_1  \
0             0            0                 1                           0   
1             0            0                 1                           0   
2             0            0                 1                           0   
3             0            0                 1                           0   
4             0            0                 1                           0   

   group_size_2 * homeowner_1  group_size_3 * homeowner_1  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
4                           0                           0   

   group_size_4 * homeowner_0  group_size_4 * homeowner_1  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
4                           0                           0   

   homeowner_0 * married_couple_1  homeowner_1 * married_couple_0  \
0                               1                               0   
1                               1                               0   
2                               1                               0   
3                               1                               0   
4                               1                               0   

   homeowner_1 * married_couple_1  
0                               0  
1                               0  
2                               0  
3                               0  
4                               0

In [76]:
import itertools
# e)
combination = []
group_size = data['group_size'].unique().tolist()
group_size.sort()
combination.append(group_size)
combination.append(data['homeowner'].unique().tolist())
combination.append(data['married_couple'].unique().tolist())
print('Combinations list', combination)
X = list(itertools.product(*combination))
combination = []
for i in X:
    combination.append(i)
    
df = pd.DataFrame(combination,columns=['group_size','homeowner','married_couple'])#index=[i for i in range(1,17)])
df

Combinations list [[1, 2, 3, 4], [0, 1], [1, 0]]


group_size  homeowner  married_couple
0            1          0               1
1            1          0               0
2            1          1               1
3            1          1               0
4            2          0               1
5            2          0               0
6            2          1               1
7            2          1               0
8            3          0               1
9            3          0               0
10           3          1               1
11           3          1               0
12           4          0               1
13           4          0               0
14           4          1               1
15           4          1               0

In [61]:
len(designX.columns)

21

In [62]:
df

group_size  homeowner  married_couple  group_size_1 * homeowner_0  \
0            1          0               1                           1   
1            1          0               0                           1   
2            1          1               1                           0   
3            1          1               0                           0   
4            2          0               1                           0   
5            2          0               0                           0   
6            2          1               1                           0   
7            2          1               0                           0   
8            3          0               1                           0   
9            3          0               0                           0   
10           3          1               1                           0   
11           3          1               0                           0   
12           4          0               1                           0   
13           4          0               0                           0   
14           4          1               1                           0   
15           4          1               0                           0   

    group_size_1 * homeowner_1  group_size_2 * homeowner_0  \
0                            0                           0   
1                            0                           0   
2                            1                           0   
3                            1                           0   
4                            0                           1   
5                            0                           1   
6                            0                           0   
7                            0                           0   
8                            0                           0   
9                            0                           0   
10                           0                           0   
11                           0                           0   
12                           0                           0   
13                           0                           0   
14                           0                           0   
15                           0                           0   

    group_size_2 * homeowner_1  group_size_3 * homeowner_0  \
0                            0                           0   
1                            0                           0   
2                            0                           0   
3                            0                           0   
4                            0                           0   
5                            0                           0   
6                            1                           0   
7                            1                           0   
8                            0                           1   
9                            0                           1   
10                           0                           0   
11                           0                           0   
12                           0                           0   
13                           0                           0   
14                           0                           0   
15                           0                           0   

    group_size_3 * homeowner_1  group_size_4 * homeowner_0  \
0                            0                           0   
1                            0                           0   
2                            0                           0   
3                            0                           0   
4                            0                           0   
5                            0                           0   
6                            0                           0   
7                            0                           0   
8                            0                           0   
9                            0  

In [106]:
# Create the columns for the group_size * homeowner interaction effect
x1 = pd.get_dummies(df[['group_size']].astype('category'))
x2 = pd.get_dummies(df[['homeowner']].astype('category'))
x3 = pd.get_dummies(df[['married_couple']].astype('category'))

y = data['A'].astype('category')
y_ = pd.DataFrame(y.where(y.isnull(), 1))


t = x1.join(x2)
t = t.join(x3)
t = t.join(create_interaction(x1,x2))
t = t.join(create_interaction(x2,x3))
t = t.join(y_)
# print(t.shape)
# print(t)
# x12 = create_interaction(x1, x2)

logit = sm.MNLogit(y, designX)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
pred = thisFit.predict(t.astype('float'))
# pd.DataFrame(pred)

         Current function value: 0.889553
         Iterations: 100


D:\Anaconda\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [108]:
pd.DataFrame(pred) #e)

0         1         2
0   0.232693  0.631125  0.136182
1   0.339172  0.526404  0.134424
2   0.291708  0.558614  0.149678
3   0.277631  0.571913  0.150455
4   0.211361  0.593611  0.195028
5   0.309408  0.497251  0.193341
6   0.273431  0.528846  0.197723
7   0.260127  0.541208  0.198666
8   0.231448  0.570103  0.198450
9   0.334430  0.471382  0.194188
10  0.247842  0.563490  0.188669
11  0.235309  0.575504  0.189187
12  0.329440  0.439438  0.231122
13  0.446750  0.340999  0.212251
14  0.230644  0.569701  0.199654
15  0.218756  0.581247  0.199997

In [98]:
t

group_size_1  group_size_2  group_size_3  group_size_4  homeowner_0  \
0              1             0             0             0            1   
1              1             0             0             0            1   
2              1             0             0             0            0   
3              1             0             0             0            0   
4              0             1             0             0            1   
5              0             1             0             0            1   
6              0             1             0             0            0   
7              0             1             0             0            0   
8              0             0             1             0            1   
9              0             0             1             0            1   
10             0             0             1             0            0   
11             0             0             1             0            0   
12             0             0             0             1            1   
13             0             0             0             1            1   
14             0             0             0             1            0   
15             0             0             0             1            0   

    homeowner_1  married_couple_0  married_couple_1  \
0             0                 0                 1   
1             0                 1                 0   
2             1                 0                 1   
3             1                 1                 0   
4             0                 0                 1   
5             0                 1                 0   
6             1                 0                 1   
7             1                 1                 0   
8             0                 0                 1   
9             0                 1                 0   
10            1                 0                 1   
11            1                 1                 0   
12            0                 0                 1   
13            0                 1                 0   
14            1                 0                 1   
15            1                 1                 0   

    group_size_1 * homeowner_0  group_size_1 * homeowner_1  ...  \
0                            1                           0  ...   
1                            1                           0  ...   
2                            0                           1  ...   
3                            0                           1  ...   
4                            0                           0  ...   
5                            0                           0  ...   
6                            0                           0  ...   
7                            0                           0  ...   
8                            0                           0  ...   
9                            0                           0  ...   
10                           0                           0  ...   
11                           0                           0  ...   
12                           0                           0  ...   
13                           0                           0  ...   
14                           0                           0  ...   
15                           0                           0  ...   

    group_size_2 * homeowner_1  group_size_3 * homeowner_0  \
0                            0                           0   
1                            0                           0   
2                            0                           0   
3                            0                           0   
4                            0                           0   
5                            0                           0   
6                            1                           0   
7                            1                           0   
8                            0                           1   
9                            0  